In [5]:
import json
import os

with open('../data/category.json', 'r') as f:
    classes = json.load(f)

labels_dir = 'labels/test'

for file in os.listdir(labels_dir):
    if file.endswith('.txt'):
        file_path = os.path.join(labels_dir, file)

        with open(file_path, 'r') as f:
            lines = f.readlines()

            filtered_lines = [
                line for line in lines
                if line.strip() and int(line.split()[0]) in classes
            ]

        with open(file_path, 'w') as f:
            f.writelines(filtered_lines)


In [4]:
import yaml

data  = """
names:
  0: shirt, blouse
  1: top, t-shirt, sweatshirt
  2: sweater
  3: cardigan
  4: jacket
  5: vest
  6: pants
  7: shorts
  8: skirt
  9: coat
  10: dress
  11: jumpsuit
  12: cape
  13: glasses
  14: hat
  15: headband, head covering, hair accessory
  16: tie
  17: glove
  18: watch
  19: belt
  21: tights, stockings
  22: sock
  23: shoe
  24: bag, wallet
  25: scarf
  27: hood
  38: bow
  39: flower
  43: ruffle

path: /kaggle/input/fashionpedia-dataset
test: images/test
train: images/train
val: images/test
"""

with open('data2.yaml', 'w') as f:
    yaml.dump(yaml.safe_load(data), f)

In [9]:
import os
import json

with open('../data/category.json', 'r') as f:
    classes = json.load(f)

remap_dict = {old: new for new, old in enumerate(classes)}

labels_dir = 'labels/val'

for filename in os.listdir(labels_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(labels_dir, filename)

        with open(file_path, "r") as file:
            lines = file.readlines()

        remapped_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            old_class_id = int(parts[0])
            new_class_id = remap_dict.get(old_class_id)
            if new_class_id is not None:
                remapped_line = " ".join([str(new_class_id)] + parts[1:]) + "\n"
                remapped_lines.append(remapped_line)

        # Overwrite file with remapped labels
        with open(file_path, "w") as file:
            file.writelines(remapped_lines)